In [1]:
## setup the environment
# !wget -O prepare_corpus.py https://raw.githubusercontent.com/tianyizz/project-reddit_watson/master/prepare_corpus.py
# !wget -O urlmarker.py https://raw.githubusercontent.com/tianyizz/project-reddit_watson/master/urlmarker.py
# !pip install watson_developer_cloud
!pip install ibm_watson

     |████████████████████████████████| 368kB 13.9MB/s eta 0:00:01
     |████████████████████████████████| 204kB 29.6MB/s eta 0:00:01
  Stored in directory: /home/dsxuser/.cache/pip/wheels/b7/10/b4/509307276e9d558a6b06d8e69f8faf6cfbdf2d1b83abb6b8d4
  Stored in directory: /home/dsxuser/.cache/pip/wheels/6a/42/50/f96888116b329578304f9dda4693cef6f3e76e18272d22cb6c
Successfully built ibm-watson ibm-cloud-sdk-core


In [2]:
# import types
import pandas as pd
import numpy as np
from botocore.client import Config
import ibm_boto3
# import nltk
# nltk.download("stopwords")
import json
# from prepare_corpus import *
import csv
# from watson_developer_cloud import NaturalLanguageUnderstandingV1
# import watson_developer_cloud.natural_language_understanding_v1 as ai
import logging
logging.basicConfig(filename='error.log', level=logging.ERROR)  # This defines the name of the error log file
from concurrent.futures import ThreadPoolExecutor
# from tqdm import tqdm
import time
from datetime import datetime
from ibm_watson import NaturalLanguageUnderstandingV1
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from ibm_watson.natural_language_understanding_v1 import Features, EmotionOptions, SentimentOptions

def __iter__(self): return 0

In [3]:
# @hidden_cell
# The project token is an authorization token that is used to access project resources like data sources, connections, and used by platform APIs.
from project_lib import Project
project = Project(project_id='42dd98d3-0b00-4676-809d-015d08bf8d84', project_access_token='p-9249755dbd201d74774e21b20c1c46e09b1e31c3')
pc = project.project_context

# Import all tweets from IBM Watson Project

### Get all tweets

In [4]:
project.get_file('all_tweets_with_stratify_groups-2020_02_13.csv')

[{'file_content': <_io.BytesIO at 0x7fd21adc9678>,
  'data_file_title': 'all_tweets_with_stratify_groups-2020_02_13.csv'},
 {'file_content': <_io.BytesIO at 0x7fd21adc96d0>,
  'data_file_title': 'data_profile_8729b793-18fa-464f-b2d4-49b16e5fc696'}]

In [5]:
df_all = pd.read_csv(project.get_file('all_tweets_with_stratify_groups-2020_02_13.csv')[0]['file_content'])

In [6]:
# look at the datadd
df_all.shape

(1280941, 8)

### Get data run so far

In [7]:
df_done = pd.read_csv(project.get_file('2020-02-25_NLU_Run_Incomplete.csv'))

In [8]:
df_done.shape

(1033179, 11)

### Get remaining rows

In [9]:
print("Expected number of remaining tweets is less than or equal to", df_all.shape[0] - df_done.shape[0])

Expected number of remaining tweets is less than or equal to 247762


In [19]:
df = df_all.merge(df_done, how='left', on='tweet_id', indicator='ind').query('ind=="left_only"').loc[:,:'stratify']

In [21]:
print(df.shape)
df.head()

(185716, 8)


,tweet_id,is_ira,date,year,month,twitter_account,text,stratify
1096193,800019339000840192,0,2016-11-19 16:54:08,2016,11,chicagotribune,What happened on Friday night at Hamilton in N...,350
1096194,800017750735327232,0,2016-11-19 16:47:50,2016,11,chicagotribune,Donald Trump agrees to settle 6-year-old Trump...,350
1096195,800014619347734532,0,2016-11-19 16:35:23,2016,11,chicagotribune,"English manor in Winnetka with conservatory, p...",350
1096196,800014309338337281,0,2016-11-19 16:34:09,2016,11,chicagotribune,"If a nation's 1st female president fails, does...",350
1096197,800010536955981824,0,2016-11-19 16:19:10,2016,11,chicagotribune,Grandson of U.S. Rep. Danny Davis was shot dea...,350


In [22]:
# make sure none of the rows of the tweet_ids of df appear in df_done
df.merge(df_done, how='inner', on='tweet_id')

,tweet_id,is_ira,date,year,month,twitter_account,text,stratify,nlu_status,language,n_char,sentiment_label,sentiment_score,emotion_anger,emotion_disgust,emotion_fear,emotion_joy,emotion_sadness


# TODO:

~1. combine the two sets into a single df called `df` keeping id column, text, and a column for whether it was control or IRA.~
2. modify below code to work with `df`

~3. change save location to google drive rather than ibm cloud~
4. change string names to backup data in appropriate cloud object store bucket use "russiantrolls-donotdelete-pr-f0fvpzx7cjymub" bucket
5. Update api keys
6. Remove unnecessary code
7. Add comments

# Set up IBM NLU credentials

In [23]:
# note: test using lite-tier service 
# # use standard tier for final run

authenticator = IAMAuthenticator("uBuJNuYBAU5iKzmQ-8eigmM3AL2tT0UcNCcbbV1lOxa4")
service = NaturalLanguageUnderstandingV1(
    version='2019-07-12',
    authenticator=authenticator
)

# Use Watson NLU

### Define functions to make NLU call and process results

In [24]:
# get response from NLU
def get_response(x):
    try:
        resp = service.analyze(text=x.text,
                                   features=Features(
                                       sentiment=SentimentOptions(),
                                       emotion=EmotionOptions())
                                   ).get_result()
    except Exception as e:
        resp = None
        print("Something went wrong.... Error: ", e)
        logging.error(e)  # This should log the error messages to an error.log file

    return resp

# prepare the response for entry into a dataframe
def condition_response(resp):
    if resp is None:        
        cr = dict(nlu_status = "fail",
                emotion_anger = None,
                emotion_disgust = None,
                emotion_fear = None,
                emotion_joy = None,
                emotion_sadness = None,
                sentiment_label = None,
                sentiment_score = None,
                language = None,
                n_char = None)
    else:
        emotions = None if resp.get('emotion') is None else resp['emotion']['document']['emotion']
        sentiments = None if resp.get('sentiment') is None else resp['sentiment']['document']
        
        cr = dict(nlu_status = "pass",
                emotion_anger =  None if emotions is None else emotions['anger'],
                emotion_disgust = None if emotions is None else emotions['disgust'],
                emotion_fear = None if emotions is None else emotions['fear'],
                emotion_joy = None if emotions is None else emotions['joy'],
                emotion_sadness = None if emotions is None else emotions['sadness'],
                sentiment_label = None if sentiments is None else sentiments['label'],
                sentiment_score = None if sentiments is None else sentiments['score'],
                language = resp.get('language'),
                n_char = None if resp.get('usage') is None else resp['usage']['text_characters'])
        
    return cr

# process an individual row
def process_row(x):
    rowIndex = x.name
    
    cr = condition_response(get_response(x))
    cr['tweet_id'] = x['tweet_id']
    
    return cr

In [25]:
# define a function to do this nicely:
def get_results_multi_thread(df, nthread=30):
    
    # get an iterable for the rows of the dataframe
    rows = df.iterrows()
    nrows = df.shape[0]

    # wrapper function to handle extra index in iterable
    def process_row_no_idx(row):
        return process_row(row[1])
    
    # send rows to Watson NLU asynchronously
    with ThreadPoolExecutor(max_workers=nthread) as executor:
        # list(tqdm(executor.map(f, my_iter), total=len(my_iter)))
        futures = executor.map(process_row_no_idx, rows)

    # turn above iterator into list of results
    list_results = []
    for r in futures:
        list_results.append(r)

    # turn the list into a dataframe
    df_results = pd.DataFrame(list_results, columns=['tweet_id', 
                                                       'nlu_status', 
                                                       'language',
                                                       'n_char', 
                                                       'sentiment_label', 
                                                       'sentiment_score', 
                                                       'emotion_anger', 
                                                       'emotion_disgust', 
                                                       'emotion_fear', 
                                                       'emotion_joy', 
                                                       'emotion_sadness'])
    
    # return the results
    return df_results

In [26]:
get_results_multi_thread(df.sample(100), nthread=20)

,tweet_id,nlu_status,language,n_char,sentiment_label,sentiment_score,emotion_anger,emotion_disgust,emotion_fear,emotion_joy,emotion_sadness
0,794167545024516096,pass,en,96,neutral,0.000000,0.068878,0.057865,0.040157,0.482309,0.410321
1,774012772996702208,pass,en,120,negative,-0.770034,0.171190,0.106167,0.284293,0.047646,0.547382
2,797343012598620160,pass,en,152,neutral,0.000000,0.432504,0.315169,0.131000,0.127086,0.150663
3,796820587683414017,pass,en,102,neutral,0.000000,0.255253,0.150752,0.261614,0.213707,0.142413
4,794563130487689218,pass,en,136,negative,-0.620385,0.230370,0.543884,0.017471,0.322203,0.065525
5,777647233856602112,pass,en,81,neutral,0.000000,0.049137,0.017591,0.092779,0.167702,0.693816
6,776753000853438465,pass,en,126,neutral,0.000000,0.055188,0.046668,0.075542,0.637108,0.216956
7,775156348992487425,pass,en,104,neutral,0.000000,0.359269,0.526384,0.053707,0.099352,0.228166
8,781129744419020800,pass,en,95,neutral,0.000000,0.042213,0.095400,0.054091,0.688184,0.110841
9,796262323962810368,pass,en,100,neutral,0.000000,0.200748,0.124504,0.183864,0.017699,0.655180


In [27]:
186000/3000

62.0

In [ ]:
#TODO: change COS bucket/names

num_splits = 60
#df_test = df.sample(3000, replace=False)
#df_split = np.array_split(df_test, num_splits)
df_split = np.array_split(df, num_splits)

now = datetime.now()

split_idx = 0
loop_times = []
cos_res = []

total_start = time.time()

for d in df_split:
    start_time = time.time()
    # progress
    print("Processing %i rows for split %i of %i"  % (d.shape[0], split_idx+1, num_splits))
    
    try:
        # get nlu results
        d_res = get_results_multi_thread(d, nthread=40)

        # write to file
        filename = 'df_trolls_'+ now.strftime("%Y-%m-%d--%H-%M-%S") + str(split_idx).zfill(4) + '.csv' 
        project.save_data(filename, data=d_res.to_csv(index=False), overwrite=False)
    except Exception as e:
        print("Failed to process split %i. Exception: %s" % (split_idx, e))
        filename_err = 'err_df_trolls_' + now.strftime("%Y-%m-%d--%H-%M-%S") + str(split_idx).zfill(4) + '.csv' 
        print("Saving failed split data to cos bucket. Key: %s" % filename_err)
        
        #project.save_data(filename_err, data=d.to_csv(index=False), set_project_asset=True, overwrite=True)
        project.save_data(filename_err, data=d.to_csv(index=False), overwrite=False)
    
    duration = time.time() - start_time
    total_time = time.time() - total_start
    loop_times.append(duration)
    avg_loop_time = sum(loop_times) / len(loop_times)
    est_remain = (num_splits - split_idx - 1)*avg_loop_time / 60
    
    # Show progress
    print(" --Iter. time: %0.1f seconds\n" % duration,
          "--Avg. iter. time: %0.1f seconds\n" % avg_loop_time,
          "--Est. time remaining: %0.1f minutes\n" % est_remain,
         "--Total time: %0.1f seconds\n" % total_time)
    
    split_idx+=1


Processing 3096 rows for split 1 of 60
Something went wrong.... Error:  Error: not enough text for language id, Code: 422 , X-global-transaction-id: 0e96ade5502f703cd1413d5fc7a182cc
Something went wrong.... Error:  Error: <HTML><HEAD>
<TITLE>Service Unavailable</TITLE>
</HEAD><BODY>
<H1>Service Unavailable - Zero size object</H1>
The server is temporarily unable to service your request.  Please try again
later.<P>
Reference&#32;&#35;15&#46;8e7f3517&#46;1582662830&#46;37140949
</BODY></HTML>
, Code: 503
Something went wrong.... Error:  Error: unsupported text language: unknown, Code: 400 , X-global-transaction-id: e9cdc1012871ad2f03e833e535609bce
Something went wrong.... Error:  Error: unsupported text language: la, Code: 400 , X-global-transaction-id: 5d7d7926bde2e7c6c611fc6c65a77ffd
 --Iter. time: 106.4 seconds
 --Avg. iter. time: 106.4 seconds
 --Est. time remaining: 104.6 minutes
 --Total time: 106.4 seconds

Processing 3096 rows for split 2 of 60
Something went wrong.... Error:  Er

Something went wrong.... Error:  Error: unsupported text language: unknown, Code: 400 , X-global-transaction-id: dc843e4fbf62ea7909fd9c42957d7ddc
Something went wrong.... Error:  Error: not enough text for language id, Code: 422 , X-global-transaction-id: ece82bff75966bfefa07018ebccadd89
 --Iter. time: 110.0 seconds
 --Avg. iter. time: 109.4 seconds
 --Est. time remaining: 72.9 minutes
 --Total time: 2187.7 seconds

Processing 3095 rows for split 21 of 60
 --Iter. time: 108.5 seconds
 --Avg. iter. time: 109.3 seconds
 --Est. time remaining: 71.1 minutes
 --Total time: 2296.2 seconds

Processing 3095 rows for split 22 of 60
 --Iter. time: 94.9 seconds
 --Avg. iter. time: 108.7 seconds
 --Est. time remaining: 68.8 minutes
 --Total time: 2391.1 seconds

Processing 3095 rows for split 23 of 60
 --Iter. time: 96.7 seconds
 --Avg. iter. time: 108.2 seconds
 --Est. time remaining: 66.7 minutes
 --Total time: 2487.8 seconds

Processing 3095 rows for split 24 of 60
Something went wrong.... Erro

Something went wrong.... Error:  Error: unsupported text language: unknown, Code: 400 , X-global-transaction-id: d8168430bd65ef60c51c9deb04cff30d
Something went wrong.... Error:  Error: unsupported text language: unknown, Code: 400 , X-global-transaction-id: 82042711eda6dfa6779507da86a9b168
 --Iter. time: 106.1 seconds
 --Avg. iter. time: 106.8 seconds
 --Est. time remaining: 35.6 minutes
 --Total time: 4272.8 seconds

Processing 3095 rows for split 41 of 60
 --Iter. time: 102.7 seconds
 --Avg. iter. time: 106.7 seconds
 --Est. time remaining: 33.8 minutes
 --Total time: 4375.4 seconds

Processing 3095 rows for split 42 of 60
Something went wrong.... Error:  Error: not enough text for language id, Code: 422 , X-global-transaction-id: 9e5d3870298645bf39de68dddb5f6316
 --Iter. time: 102.1 seconds
 --Avg. iter. time: 106.6 seconds
 --Est. time remaining: 32.0 minutes
 --Total time: 4477.5 seconds

Processing 3095 rows for split 43 of 60
Something went wrong.... Error:  Error: unsupported 